In [1]:
import openrouteservice as ors
import uuid
import folium
from IPython.core.display import display, HTML

colors = ['#191970','#006400', '#ff4500', '#ffd700', '#00ff00', '#0000ff', '#b0c4de', '#ff1493']

m = folium.Map(location = [49.411085, 8.685894], zoom_start = 13)

locs = [[8.681495,49.41461],[8.686507,49.41943]]
body = {"locations":locs,"range":[300,200]}#,"intersections":"true"}


host = "http://localhost:8082/ors"
#host = "http://api.openrouteservice.org"
key = "5b3ce3597851110001cf6248554c879acee241f6bc733402d2d351be"
endpoint = "isochrones"
profile = "driving-car"
formats = "geojson"

# set up client
client = ors.Client(base_url=host, key = key)
resp = client.request(url = f'/v2/{endpoint}/{profile}', requests_kwargs = {"timeout":None}, get_params={}, post_json = body)

for isochrone in resp['features']:
    center = [round(x, 6) for x in isochrone['properties']['center']]
    i = locs.index(center)
    folium.Polygon(locations=[list(reversed(coord)) for coord in isochrone['geometry']['coordinates'][0]],
                  fill_color=colors[i],
                  #popup=folium.Popup("Population: {} people".format(isochrone['properties']['total_pop'])),
                  opacity=0.5).add_to(m)
    folium.Circle(location = list(reversed(center)), color=colors[i], radius=6).add_to(m)

m

In [56]:
m = folium.Map(location = [49.411085, 8.685894], zoom_start = 13)
n = folium.Map(location = [49.411085, 8.685894], zoom_start = 13)

body = {"locations":locs,"range":[300,200],"intersections":"true"}

intersection_resp = client.request(url = f'/v2/{endpoint}/{profile}', requests_kwargs = {"timeout":None}, get_params={}, post_json = body)

for isochrone in intersection_resp['features']:
    print(isochrone['properties'])
    # do stuff with isochrones
    if (len(isochrone['properties']) == 3):
        center = [round(x, 6) for x in isochrone['properties']['center']]
        i = intersection_resp['features'].index(isochrone)
        folium.Polygon(locations=[list(reversed(coord)) for coord in isochrone['geometry']['coordinates'][0]],
                      fill_color=colors[i],
                      #popup=folium.Popup("Population: {} people".format(isochrone['properties']['total_pop'])),
                      opacity=0.5).add_to(m)
        folium.Circle(location = list(reversed(center)), color=colors[i], radius=6).add_to(m)
        
    # do stuff with contours
    if (len(isochrone['properties']) == 1):
        folium.Polygon(locations=[list(reversed(coord)) for coord in isochrone['geometry']['coordinates'][0]],
                      fill_color='red',
                      popup=folium.Popup(f"contours:{isochrone['properties']['contours']}"),
                      opacity=0.5).add_to(n)

m

{'group_index': 0, 'value': 200.0, 'center': [8.681494738985997, 49.4146099952023]}
{'group_index': 0, 'value': 300.0, 'center': [8.681494738985997, 49.4146099952023]}
{'group_index': 1, 'value': 200.0, 'center': [8.686506699153233, 49.419429973599385]}
{'group_index': 1, 'value': 300.0, 'center': [8.686506699153233, 49.419429973599385]}
{'contours': [[0, 0], [1, 0]]}
{'contours': [[0, 0], [1, 1]]}
{'contours': [[0, 1], [1, 0]]}
{'contours': [[0, 1], [1, 1]]}


In [55]:
n

In [65]:
import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
        document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)
        
RenderJSON(intersection_resp)

In [44]:
body = {"locations":locs,"range":[300,200]}
resp = client.request(url = f'/v2/{endpoint}/{profile}', requests_kwargs = {"timeout":None}, get_params={}, post_json = body)
RenderJSON(resp)

In [45]:
body = {"locations":locs,"range":[300,200],"intersections":"true"}
intersection_resp = client.request(url = f'/v2/{endpoint}/{profile}', requests_kwargs = {"timeout":None}, get_params={}, post_json = body)

RenderJSON(intersection_resp)

In [63]:
locs = [[8.681495,49.41461],[8.686507,49.41943]]

body = {"locations":locs,"range":[300], "interval":20}#,"intersections":"true"}

resp = client.request(url = f'/v2/{endpoint}/{profile}', requests_kwargs = {"timeout":None}, get_params={}, post_json = body)

RenderJSON(resp)

ApiError: 400 ({'error': {'code': 3012, 'message': "Parameter 'interval' is out of range: Resulting number of 15 isochrones exceeds maximum value of 10."}, 'info': {'engine': {'version': '6.4.1', 'build_date': '2021-04-08T15:56:10Z'}, 'timestamp': 1617897390016}})